<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-05 13:15:59
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.36 C
-------------------
Today PnL: -58.16 K (-0.43%)
Current PnL: -18.98 L (-12.91%)
CY Booked + Current PnL: -10.71 L (-7.28%)
-------------------
Total profit:  2.43 L
Total loss:  -21.41 L
-------------------
Total Booked + Current PnL: 16.27 L (13.55%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.09%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 77.27 L (56.86%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.12%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-20.73,H-LC,5.16,109280.0,4968.0,12567.0,-0.68,4.76,11.50,16.81,10.0,0.40,0.83,21.57,X5K,ATH,METALS
27,DIXON,14367.00,18940.15,-26.42,H-MC,6.59,135408.0,20472.0,16114.0,-0.45,17.81,11.90,31.83,85.0,1.27,1.02,52.39,X40N,ATH,IT
67,SBIN,760.30,863.00,-12.46,M-LC,4.01,212411.0,10932.0,16292.0,0.74,5.43,7.67,13.51,62.0,0.67,1.60,18.71,XY25,NTT,BANKS
71,SIS,477.00,477.00,2269.43,H-SC,6.90,63750.0,-17340.0,17340.0,-0.87,-21.38,27.20,0.00,157.0,-1.00,0.48,29.20,OX40N,NTT,MISC
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.34,153955.0,27755.0,18428.0,0.20,21.99,11.97,36.59,156.0,1.51,1.16,37.26,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,PGHH,13388.00,18062.58,-19.82,H-MC,6.86,202320.0,1500.0,68627.0,-0.87,0.75,33.92,34.92,84.0,0.02,1.53,5.46,X40,ATH,FMCG
91,WIPRO,243.46,420.00,-11.03,M-LC,5.51,152266.0,1321.0,108139.0,-0.19,0.88,71.02,72.51,55.0,0.01,1.15,6.95,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.16,H-LC,1.81,204774.0,274.0,41139.0,-1.35,0.13,20.09,20.25,9.0,0.01,1.55,8.88,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-16.90,H-LC,2.81,164968.0,-36418.0,124039.0,-0.97,-18.08,75.19,43.51,19.0,-0.29,1.25,14.83,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.08,H-LC,3.14,182574.0,-17736.0,77722.0,-0.34,-8.85,42.57,29.94,24.0,-0.23,1.38,26.13,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-9.80,H-LC,3.43,187190.0,-13316.0,62091.0,0.00,-6.64,33.17,24.33,30.0,-0.21,1.41,7.94,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-16.78,H-LC,4.28,263870.0,-1556.0,58474.0,-0.97,-0.59,22.16,21.44,26.0,-0.03,1.99,6.02,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-23.54,H-SC,6.60,217600.0,24580.0,23457.0,3.23,12.73,10.78,24.89,128.0,1.05,1.64,38.79,X40N,NTT,HEALTHCARE
44,INDUSINDBK,1354.50,1800.00,69.16,L-MC,8.01,45831.0,-31376.0,56771.0,2.60,-40.64,123.87,32.89,258.0,-0.55,0.35,26.21,XR,NTT,BANKS
87,VALIANTORG,512.64,838.00,-321.46,H-SC,20.20,94172.0,-39114.0,123704.0,2.13,-29.35,131.36,63.47,147.0,-0.32,0.71,57.86,XR,NTT,CHEMICALS
69,SHALBY,261.39,327.00,976.56,M-SC,8.85,146431.0,-34712.0,80186.0,1.71,-19.16,54.76,25.10,233.0,-0.43,1.11,17.35,XY24,NTT,HEALTHCARE
70,SIEMENS,4406.90,7969.85,-10.00,H-LC,36.96,76065.0,-34107.0,123180.0,1.64,-30.96,161.94,80.85,29.0,-0.28,0.57,12.03,AR,ATH,ELECTRICAL


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,5551.76,7934.00,-44.34,H-LC,8.00,198608.0,-17911.0,110823.0,-5.48,-8.27,55.80,42.91,5.0,-0.16,1.50,3.81,AR,NTT,ELECTRICAL
75,SYMPHONY,1306.42,1306.00,-37.35,M-SC,9.01,133581.0,-37560.0,37510.0,-3.26,-21.95,28.08,-0.03,197.0,-1.00,1.01,0.00,OX40N,NTT,DURABLES
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.44,133830.0,10494.0,123271.0,-2.84,8.51,92.11,108.45,118.0,0.09,1.01,32.76,AR,ATH,MISC
86,VAIBHAVGBL,350.21,521.00,44.35,H-SC,7.05,96483.0,-55508.0,129635.0,-2.66,-36.52,134.36,48.77,127.0,-0.43,0.73,11.52,XR,NTT,JEWELLERY
76,TANLA,917.30,1963.11,-35.72,H-SC,6.84,157161.0,-82254.0,355215.0,-2.39,-34.36,226.02,114.01,133.0,-0.23,1.19,44.14,AR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,125.45,H-SC,5.09,147870.0,-26689.0,26720.0,-0.13,-15.29,18.07,0.02,135.0,-1.00,1.12,28.23,OX40N,NTT,PAINTS
83,UJJIVANSFB,52.77,53.0,36.52,H-SC,20.71,118719.0,-23760.0,24385.0,0.05,-16.68,20.54,0.44,163.0,-0.97,0.90,40.12,OX40N,NTT,BANKS
82,TTKPRESTIG,769.29,770.0,-10.01,H-SC,11.82,82261.0,-18516.0,18607.0,-0.44,-18.37,22.62,0.09,145.0,-1.00,0.62,8.07,OX40N,NTT,DURABLES
77,TATAELXSI,7332.28,7332.0,-20.17,H-MC,1.97,83776.0,-18876.0,18875.0,-0.78,-18.39,22.53,-0.00,101.0,-1.00,0.63,27.35,OX40N,NTT,IT
49,KANSAINER,299.63,340.0,-68.75,H-SC,1.84,217809.0,-51858.0,88191.0,-0.80,-19.23,40.49,13.47,136.0,-0.59,1.65,10.16,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.06,H-LC,14.54,238126.0,-57808.0,151710.0,-0.70,-19.53,63.71,31.73,1.0,-0.38,1.80,1.66,X40,BTT,IT
88,VBL,492.64,671.64,-10.17,H-LC,11.41,272578.0,7538.0,88751.0,-0.38,2.84,32.56,36.33,3.0,0.08,2.06,16.52,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-18.52,H-LC,8.22,261000.0,-601.0,146212.0,-1.51,-0.23,56.02,55.67,4.0,-0.00,1.97,5.86,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.34,H-LC,8.00,198608.0,-17911.0,110823.0,-5.48,-8.27,55.80,42.91,5.0,-0.16,1.50,3.81,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,1.85,H-LC,8.61,180536.0,-10110.0,56616.0,0.37,-5.30,31.36,24.40,8.0,-0.18,1.36,9.79,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,20.18,H-SC,13.34,153955.0,27755.0,18428.0,0.2,21.99,11.97,36.59,156.0,1.51,1.16,37.26,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,31.89,H-SC,13.44,134921.0,9624.0,49449.0,-0.76,7.68,36.65,47.15,144.0,0.19,1.02,68.79,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.34,153955.0,27755.0,18428.0,0.20,21.99,11.97,36.59,156.0,1.51,1.16,37.26,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-0.95,H-MC,4.59,159586.0,8673.0,48626.0,0.52,5.75,30.47,37.97,90.0,0.18,1.21,32.51,AR,ATH,AUTO
19,CAMS,3643.00,5226.82,-5.47,H-SC,1.42,106154.0,4150.0,40201.0,-0.20,4.07,37.87,43.48,120.0,0.10,0.80,21.53,X40N,ATH,MISC
25,DABUR,505.20,735.00,-5.75,H-MC,5.12,204068.0,7040.0,82586.0,-1.21,3.57,40.47,45.49,100.0,0.09,1.54,17.08,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,31.89,H-SC,13.44,134921.0,9624.0,49449.0,-0.76,7.68,36.65,47.15,144.0,0.19,1.02,68.79,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,-0.95,H-MC,4.59,159586.0,8673.0,48626.0,0.52,5.75,30.47,37.97,90.0,0.18,1.21,32.51,AR,ATH,AUTO
90,WHIRLPOOL,1219.98,2270.00,-41.46,M-SC,6.24,98385.0,6886.0,71860.0,-1.88,7.53,73.04,86.07,211.0,0.10,0.74,42.32,XR,NTT,DURABLES
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.44,133830.0,10494.0,123271.0,-2.84,8.51,92.11,108.45,118.0,0.09,1.01,32.76,AR,ATH,MISC
91,WIPRO,243.46,420.00,-11.03,M-LC,5.51,152266.0,1321.0,108139.0,-0.19,0.88,71.02,72.51,55.0,0.01,1.15,6.95,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2269.43,H-SC,6.90,63750.0,-17340.0,17340.0,-0.87,-21.38,27.20,0.00,157.0,-1.00,0.48,29.20,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.58,67795.0,-45754.0,85801.0,0.37,-40.29,126.56,35.27,267.0,-0.53,0.51,65.93,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-20.05,H-SC,16.48,90885.0,-7095.0,28965.0,-0.31,-7.24,31.87,22.32,151.0,-0.24,0.69,31.81,XR,ATH,FINANCE
3,ALKYLAMINE,2347.98,4546.37,2.72,H-SC,19.26,95628.0,-5335.0,99864.0,-1.56,-5.28,104.43,93.63,149.0,-0.05,0.72,42.44,SR,ATH,CHEMICALS
36,HINDALCO,651.95,761.55,-20.73,H-LC,5.16,109280.0,4968.0,12567.0,-0.68,4.76,11.50,16.81,10.0,0.40,0.83,21.57,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SHALBY,261.39,327.00,976.56,M-SC,8.85,146431.0,-34712.0,80186.0,1.71,-19.16,54.76,25.10,233.0,-0.43,1.11,17.35,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.54,H-SC,6.60,217600.0,24580.0,23457.0,3.23,12.73,10.78,24.89,128.0,1.05,1.64,38.79,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.34,153955.0,27755.0,18428.0,0.20,21.99,11.97,36.59,156.0,1.51,1.16,37.26,XY24,NTT,HEALTHCARE
27,DIXON,14367.00,18940.15,-26.42,H-MC,6.59,135408.0,20472.0,16114.0,-0.45,17.81,11.90,31.83,85.0,1.27,1.02,52.39,X40N,ATH,IT
88,VBL,492.64,671.64,-10.17,H-LC,11.41,272578.0,7538.0,88751.0,-0.38,2.84,32.56,36.33,3.0,0.08,2.06,16.52,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.68
1,25,41.53
2,50,71.34


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.70
LC    37.07
MC    22.19
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.06
X40N     15.20
X40      13.23
XY25     11.72
XR        8.97
OX40N     7.70
AR        7.56
X5K       2.21
MH        1.77
X200      1.38
SR        1.16
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.99
H-SC    27.27
H-MC    19.21
M-SC    12.01
M-LC     7.02
M-MC     2.63
L-SC     1.42
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.28,-1.90,34.66
IT,11.93,-18.02,75.35
FINANCE,8.38,-17.64,60.33
BANKS,7.45,-15.50,62.63
PAINTS,5.77,-14.58,37.16
MISC,5.15,-19.65,89.65
HEALTHCARE,5.12,3.16,21.29
ELECTRICAL,4.70,-12.81,61.66
AUTO,4.39,-16.30,66.89


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2790641.0,23
XR,1105503.0,13
AR,1063778.0,9
X40,819075.0,10
X40N,594582.0,12
XY25,499178.0,8
OX40N,358010.0,10
SR,189856.0,2
X5K,147183.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2711740.0,30
M-SC,1379290.0,15
H-LC,1315953.0,20
H-MC,1162883.0,15
M-LC,447902.0,5
M-MC,330748.0,2
L-SC,272852.0,3
L-MC,56771.0,1
L-LC,49191.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      934173.0      7
M-SC       XY24      840043.0      7
H-SC       AR        624387.0      4
H-LC       X40       546297.0      6
H-SC       XR        514701.0      6
H-MC       XY24      458733.0      4
M-MC       XY24      330748.0      2
H-LC       X40N      302002.0      6
           AR        234003.0      2
H-SC       X40N      214614.0      4
H-MC       XY25      207999.0      2
           X40       196942.0      3
H-SC       SR        189856.0      2
M-LC       XY24      188855.0      2
H-SC       OX40N     184560.0      6
L-SC       XR        184454.0      2
M-SC       AR        156762.0      2
M-LC       X5K       134616.0      1
H-MC       XR        121389.0      1
M-SC       XY25      120423.0      1
           XR        120049.0      2
M-LC       XR        108139.0      1
H-LC       XY25      105273.0      3
L-SC       OX40N      88398.0      1
H-MC       X40N       77966.0      2
H-LC       X200       77722.0      1
M-SC       X40        75836.0      1
           OX40N      66177.0      2
L-MC       XR         56771.0      1
H-SC       MH         49449.0      1
L-LC       XY25       49191.0      1
H-MC       AR         48626.0      1
H-LC       XY24       38089.0      1
H-MC       MH         32353.0      1
           OX40N      18875.0      1
M-LC       XY25       16292.0      1
H-LC       X5K        12567.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 58.0 seconds
